In [1]:
import numpy as np

In [3]:
class Operation:
    def __init__(self,input_nodes=[]):
        self.input_nodes = input_nodes
        #Initialize list of consumers
        self.consumers=[]
        for input_node in input_nodes:
            input_node.consumers.append(self)
            
        _default_graph.operations.append(self)
        
    def compute(self):
        pass

In [4]:
#ADDITION

class add(Operation):
    "Returns x+y "
    def __init__(self,x,y):
        "Constructor add"
        "Args: x: first summand node; y: second summand node"
        super().__init__([x,y])
        
    def compute(self,x_value, y_value):
        return x_value + y_value

In [16]:
#MULTIPLICATION

class matmul(Operation):
    def __init__(self,a,b):
        super().__init__([a,b])
    def compute(self,a_value,b_value):
        return a_value.dot(b_value)

In [17]:
#PLACEHOLDERS

class placeholder:
    def __init__(self):
        self.consumers =[]
        _default_graph.placeholders.append(self)

In [18]:
#VARIABLE

class Variable:
    def __init__(self,initial_value=None):
        self.value = initial_value
        self.consumers = []
        
        _default_graph.variables.append(self)

In [19]:
class Graph:
    "Represents computational graph"
    def __init__(self):
        "Construct graph"
        self.operations = []
        self.placeholders = []
        self.variables = []
        
    def as_default(self):
        global _default_graph
        _default_graph = self

In [20]:
#EXAMPLE

Graph().as_default()

#Create Variables
A = Variable([[1,0],[0,-1]])
b = Variable([1,1])

#Create placeholder
x = placeholder()

#Create hidden node y
y = matmul(A,x)

#Create output node z
z = add(y,b)

In [21]:
import numpy as np

class Session:
    
    def run(self,operation,feed_dict={}):
        #perform post-order traversal of the graph to bring the nodes into the right order
        nodes_postorder = traverse_postorder(operation)
        
        #Iterate all nodes to determine their value
        for node in nodes_postorder:
            
            if type(node) == placeholder:
                node.output = feed_dict[node]
            elif type(node) == Variable:
                node.output = node.value
            else:
                node.inputs = [input_node.output for input_node in node.input_nodes]
                node.output = node.compute(*node.inputs)
                
            if type(node.output) == list:
                node.output = np.array(node.output)
                
        return operation.output
    
def traverse_postorder(operation):
    nodes_postorder = []
    def recurse(node):
        if isinstance(node, Operation):
            for input_node in node.input_nodes:
                recurse(input_node)
        nodes_postorder.append(node)
    
    recurse(operation)
    return nodes_postorder

In [23]:
session = Session()
output = session.run(z,{x:[1,2]})
print(output)

[ 2 -1]
